<a href="https://colab.research.google.com/github/microsoft/CameraTraps/blob/master/detection/megadetector_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

Link in case the above badge doesn't redirect you correctly: [Open in Colab](https://colab.research.google.com/github/microsoft/CameraTraps/blob/master/detection/megadetector_colab.ipynb)

This notebook replaces a previous example by [@louis030195](https://github.com/louis030195). Improvements: updated environment setup, MegaDetector model version and support for mounting Google Drive folders so you can process your own images here.

# Running MegaDetector on camera trap images using Google Colab
Put together by Alistair Stewart, Alice Springs, May 2020.
@alsnothome

For reference please read the [MegaDetector guide on GitHub](https://github.com/microsoft/CameraTraps/blob/master/megadetector.md) and check there for updates. Here we have roughly followed the steps for running under Linux.

This notebook is designed to load camera trap image files already uploaded onto Google Drive. If you don't have images already loaded onto Google Drive or just want to see a demo of MegaDetector in action, we also provide code to download some sample images.

The steps walk through copying of all of the required model and helper files to the Colab runtime and installing all the required packages. You can then connect to your Google Drive folder and process all of the images in a folder using the MegaDetector saved model. The output is saved in a JSON file - a text based database file whose format is described in this [section](https://github.com/microsoft/CameraTraps/tree/master/api/batch_processing#batch-processing-api-output-format) in the batch API user guide. The detections (as bounding boxes) can then be rendered on your images.

The Google Colab instance will only stay open for a maximum 10-12 hrs and after that it will close and any unsaved data will be lost. We recommend saving the JSON output and annotated images into your Google Drive folder for persistent storage.

## Set up the Colab instance to run on GPU processing


Navigate to Edit→Notebook Settings and select "GPU" from the Hardware Accelerator drop-down 

## Copy the model, install dependencies, set PYTHONPATH

Note: from here on you'll start seeing a mix of code. Most are Linux system commands, rather than Python. The system commands are prefixed by a shebang `!`, which tells this notebook to execute them on the command line.

### Install TensorFlow v1

TensorFlow is already installed in Colab, but our scripts are not yet compatible with the newer version of TensorFlow. 

Please follow the next three steps in sequence and do not skip any steps :) If you were not able to follow these, you can reset the runtime by going to "Runtime" in the top menu and "Factory reset runtime".


1. Uninstall the existing version of TensorFlow (this doesn't affect your other Colabs, don't worry)


In [ ]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0


2. Install the older TensorFlow version using `pip`, with GPU processing by specifying `-gpu` and version number `1.13.1`. We also install the other required Python packages that are not already in Colab - `humanfriendly` and `jsonpickle`.

In [ ]:
pip install tensorflow-gpu==1.13.1 humanfriendly jsonpickle

     |████████████████████████████████| 345.0 MB 4.4 kB/s 
     |████████████████████████████████| 86 kB 1.7 MB/s 
     |████████████████████████████████| 3.2 MB 42.0 MB/s 
     |████████████████████████████████| 367 kB 28.3 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0


3. Importantly, you now need to **re-start the runtime** of this Colab for it to start using the older version TensorFlow that we just installed.

Click on the "Runtime" option on the top menu, then "Restart runtime". After that, you can proceed with the rest of this notebook.

Let's check that we have the right version of TensorFlow (1.13.1):

In [ ]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


### Download the MegaDetector model file

Currently, v4.1 is avaialble by direct download. The link can be found in the GitHub MegaDetector readme: MegaDetector v4.1, 2020.04.27 frozen model (.pb)

In [ ]:
!wget -O /content/megadetector_v4_1_0.pb https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb

--2021-12-08 15:42:04--  https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb
Resolving lilablobssc.blob.core.windows.net (lilablobssc.blob.core.windows.net)... 52.239.159.84
Connecting to lilablobssc.blob.core.windows.net (lilablobssc.blob.core.windows.net)|52.239.159.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245590501 (234M) [application/octet-stream]
Saving to: ‘/content/megadetector_v4_1_0.pb’

/content/megadetect 100%[===================>] 234.21M  44.7MB/s    in 5.7s    

2021-12-08 15:42:10 (41.4 MB/s) - ‘/content/megadetector_v4_1_0.pb’ saved [245590501/245590501]



### Clone the two required Microsoft git repos
This will copy the latest version of the Microsoft AI for Earth "utilities" and "Camera Traps" repositories from GitHub. These make data handling and running the model easy. 

In [ ]:
!git clone https://github.com/microsoft/CameraTraps
!git clone https://github.com/microsoft/ai4eutils

Cloning into 'CameraTraps'...
remote: Enumerating objects: 12276, done.
remote: Counting objects: 100% (1264/1264), done.
remote: Compressing objects: 100% (702/702), done.
remote: Total 12276 (delta 739), reused 1023 (delta 558), pack-reused 11012
Receiving objects: 100% (12276/12276), 122.05 MiB | 27.74 MiB/s, done.
Resolving deltas: 100% (7219/7219), done.
Cloning into 'ai4eutils'...
remote: Enumerating objects: 728, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 728 (delta 187), reused 171 (delta 84), pack-reused 419
Receiving objects: 100% (728/728), 2.63 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (418/418), done.


We'll also copy the Python scripts that run the model and produce visualization of results to the working directory.

In [ ]:
!cp /content/CameraTraps/detection/run_tf_detector_batch.py .
!cp /content/CameraTraps/visualization/visualize_detector_output.py .

### Set `PYTHONPATH` to include `CameraTraps` and `ai4eutils`

Add cloned git folders to the `PYTHONPATH` environment variable so that we can import their modules from any working directory.


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/ai4eutils"
os.environ['PYTHONPATH'] += ":/content/CameraTraps"

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: /env/python:/content/ai4eutils:/content/CameraTraps


## Mount Google Drive in Colab
You can mount your Google Drive if you have sample images there to try MegaDetector on or want to save the results to your Google Drive.

Once you run the cell below, it will show a URL and a text box.

Visit that URL to choose the Google account where the images you want to process live. After you authenticate, an authorization code will be shown. Copy the authorization code to the text box here. 

Your Google Drive folders will then be mounted under `/content/drive` and can be viewed and navigated in the Files pane.

The method is described under this Colab code snippet: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA. Never give out your account username and password. Read this Colab code snippet to understand how this connection is made and authenticated. There are other ways to connect your Google Drive or upload your data if you do not find this method suitable.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Optional: Download Sample Images

We install Microsoft Azure's [`azcopy` utility](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) which we then use to download camera trap images from the [Snapshot Serengeti](http://lila.science/datasets/snapshot-serengeti) dataset hosted on [lila.science](http://lila.science).

In [ ]:
%%bash
# download azcopy
wget -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1
rm azcopy_linux.tar.gz
chmod u+x azcopy

# copy Snapshot Serengeti images to a local directory
DATASET_URL="https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped/"
SAMPLE_DIR="S1/D05/D05_R4"
SAS_TOKEN="?st=2020-01-01T00%3A00%3A00Z&se=2034-01-01T00%3A00%3A00Z&sp=rl&sv=2019-07-07&sr=c&sig=/DGPd%2B9WGFt6HgkemDFpo2n0M1htEXvTq9WoHlaH7L4%3D"
LOCAL_DIR="/content/snapshotserengeti"

./azcopy cp "${DATASET_URL}${SAMPLE_DIR}${SAS_TOKEN}" "${LOCAL_DIR}" --recursive

In [ ]:
# this should show 48 images downloaded
!ls /content/snapshotserengeti/ -R

## Optional: Download Sample Images

We install Microsoft Azure's [`azcopy` utility](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) which we then use to download camera trap images from the [Snapshot Serengeti](http://lila.science/datasets/snapshot-serengeti) dataset hosted on [lila.science](http://lila.science).

In [ ]:
%%bash
# download azcopy
wget -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1
rm azcopy_linux.tar.gz
chmod u+x azcopy

# copy Snapshot Serengeti images to a local directory
DATASET_URL="https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped/"
SAMPLE_DIR="S1/D05/D05_R4"
SAS_TOKEN="?st=2020-01-01T00%3A00%3A00Z&se=2034-01-01T00%3A00%3A00Z&sp=rl&sv=2019-07-07&sr=c&sig=/DGPd%2B9WGFt6HgkemDFpo2n0M1htEXvTq9WoHlaH7L4%3D"
LOCAL_DIR="/content/snapshotserengeti"

./azcopy cp "${DATASET_URL}${SAMPLE_DIR}${SAS_TOKEN}" "${LOCAL_DIR}" --recursive

In [ ]:
# this should show 48 images downloaded
!ls /content/snapshotserengeti/ -R

## MegaDetector batch processing

This step executes the Python script `run_tf_detector_batch.py` that we copied from the CameraTraps repo. It has three mandatory arguments and one optional:

1.   path to the MegaDetector saved model file.
2.   a folder containing images. If your images were already on Google Drive, replace `[Image_Folder]` with your folder name from Google Drive. If you are using the sample images from Snapshot Serengeti, change `images_dir` to `'/content/snapshotserengeti'`.
3.   the output JSON file location and name - replace `[Output_Folder]` with your folder name and `[output_file_name.json]` with your file name.
4.   option `--recursive` goes through all subfolders to find and process all images within.

You will need to change the image folder path and output file path, depending on your situation.

In our experience the Colab system will take ~30 seconds to intialize and load the saved MegaDetector model. It will then iterate through all of the images in the folder specified. Processing initially takes a few seconds per image and usually settles to ~1 sec per image. That is ~60 images per minute or ~3600 images per hour. Limit the number of images in your folder so that all of the processing can be completed before the Colab session ends.

If you see the error "AssertionError: output_file specified needs to end with .json" then you haven't update the output folder and file name in the line of code below properly.

In [ ]:
images_dir = '/content/drive/My Drive/TER_images'

# choose a location for the output JSON file
output_file_path = '/content/drive/My Drive/TER_results/output.json'

Here we pass the Python variable value `output_file_path` you specified above to the bash commands below using `$` (double quoting as there are spaces in this path), to run the script. This is so that we can refer to the output file path later for visualization.

In [ ]:
!python run_tf_detector_batch.py megadetector_v4_1_0.pb "$images_dir" "$output_file_path" --recursive

TensorFlow version: 1.13.1
2021-12-08 15:52:46.452483: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-08 15:52:46.464987: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-12-08 15:52:46.465279: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x556f8f069e40 executing computations on platform Host. Devices:
2021-12-08 15:52:46.465319: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-08 15:52:46.659203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-08 15:52:46.660024: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x556f8f069ce0 executing computations on platform CUDA. Devices:
2021-12-08 15:52:46.66

## Visualize batch processing script outputs

Here we use the `visualize_detector_output.py` in the `visualization` folder of the Camera Traps repo to see the output of the MegaDetector visualized on our images. It will save images annotated with the results (original images will *not* be modified) to the `[Visualization_Folder]` you specify here.

The scripts take in a number of optional parameters to control output image size and how many are sampled (if you've processed a lot of images but only want to visualize the results on a few) - take a look at the `main()` function in the script to see what other parameters are available.

In [ ]:
visualization_dir = '/content/TER_visualization'  # pick a location for annotated images

In [ ]:
!python visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.3 --images_dir "$images_dir"

detection_categories provided
Detector output file contains 11 entries.
Starting to annotate the images...
100% 11/11 [00:00<00:00, 12.74it/s]
Rendered detection results on 11 images, saved to /content/TER_visualization.


In [ ]:
import os
from PIL import Image

In [ ]:
for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  display(im)  # display() is an iPython method that comes with the notebook

Output hidden; open in https://colab.research.google.com to view.

## Next steps

Now that you have tried applying the MegaDetector on your own images and assessed its effectiveness, here are some pointers to help you take advantage of the MegaDetector to label your survey images more quickly.


### Ways to use the output JSON in my workflow 

#### 1. Timelapse

[Timelapse](http://saul.cpsc.ucalgary.ca/timelapse/pmwiki.php?n=Main.HomePage) is an open-source tool for annotating camera trap images. We have worked with the Timelapse developer to integrate the output of our API into Timelapse, so a user can:

- Select or sort images based on whether they contain animal or people or vehicles.
- View bounding boxes during additional manual annotation steps (which may speed up review...)

See our [Integration with Timelapse](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/integration/timelapse.md) page for more information.

![Screenshot showing the Timelapse application with MegaDetector output, shown as a bounding box around the detected animal](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/integration/images/tl_boxes.jpg?raw=1)


**Subsetting the output JSON into more manageable JSONs to use with Timelapse**

People using Timelapse often split up the workload and have multiple Timelapse databases per study. We wrote a simple Windows desktop app ([Camera Trap Batch API Output Manager App](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/postprocessing/CameraTrapJsonManagerApp.md)) for splitting up the output JSON.


#### 2. Separating images into folders that contain animals/people/vehicles/nothing

Some of our collaborators do their downstream labeling without Timelapse, by moving the images to separate folders containing animals/people/vehicles/nothing according to MegaDetector output. You can use the script [separate_detections_into_folders.py](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/postprocessing/separate_detections_into_folders.py) to do that if you have some familiarity with running Python scripts. This script has only one external package ("tqdm") to install - you can also follow our [installation instructions](https://github.com/microsoft/CameraTraps#initial-setup) to run the script inside a conda virtual environment. 


### Help me process millions and millions of images

We offer a [service](https://github.com/microsoft/CameraTraps/tree/master/api/batch_processing#detector-batch-processing-api-user-guide) for processing a large quantity of camera trap images using the MegaDetector. We typically call this service on behalf of ecologist users once their images are uploaded. Please see our [collaborations](https://github.com/microsoft/CameraTraps/blob/master/collaborations.md) page for more information and get in touch with us.